In [3]:
import sys
import os

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from PyQt5.QtWidgets import QMainWindow, QWidget, QGridLayout, QVBoxLayout, QHBoxLayout, QApplication, QMessageBox, QPushButton, QLabel, QSizePolicy
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
from optic.gui.base_layouts import makeLayoutLineEditLabel, makeLayoutComboBoxLabel
from optic.gui.io_layouts import makeLayoutLoadFileWidget
from optic.gui.app_setup import setupMainWindow
from optic.gui.app_style import applyAppStyle
from optic.manager import WidgetManager, ConfigManager, DataManager, ControlManager, LayoutManager, initManagers
from optic.behavior_camera.camera import CameraDevice, DisplayEngine
from optic.gui.bind_func import (
    bindFuncExit, bindFuncLoadFileWidget
)

class BehaviorCameraGUI(QMainWindow):
    """行動実験用カメラ制御GUI"""
    
    def __init__(self):
        APP_NAME = "BEHAVIOR_CAMERA"
        QMainWindow.__init__(self)
        self.widget_manager, self.config_manager, self.data_manager, self.control_manager, self.layout_manager = initManagers(
            WidgetManager(), ConfigManager(), DataManager(), ControlManager(), LayoutManager()
        )
        self.config_manager.setCurrentApp(APP_NAME)
        self.app_keys = self.config_manager.gui_defaults["APP_KEYS"]
        self.app_key_pri = self.app_keys[0]

        self.setupUI_done = False
        setupMainWindow(self, self.config_manager.gui_defaults)

        # 変数の初期化
        self.camera_device = CameraDevice()
        self.display_engine = DisplayEngine()
        self.is_capturing = False

        # UI初期化
        self.initUI()

        # カメラ初期化
        self.initCamera()
    
    def initUI(self):
        """UIの初期化"""
        # 中央ウィジェット
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        
        # メインレイアウト
        self.layout_main = QGridLayout(self.central_widget)
        
        # セクションレイアウトの配置
        # 左側上部：Config
        self.layout_main.addLayout(self.makeLayoutSectionLeftUpper(), 0, 0, 1, 1)
        # 左側下部：Capture
        self.layout_main.addLayout(self.makeLayoutSectionLeftLower(), 1, 0, 1, 1)
        # 右側全体：Image
        self.layout_main.addLayout(self.makeLayoutSectionRight(), 0, 1, 2, 1)
        
        # レイアウトの列幅比率を設定
        self.layout_main.setColumnStretch(0, 1)
        self.layout_main.setColumnStretch(1, 3)

        # setupUI
        self.setupUI()

    def setupUI(self):
        self.bindFuncAllWidget()
        self.setupUI_done = True
        # set display label
        self.display_engine.setDisplayLabel(self.widget_manager.dict_label["camera_preview"])

    def initCamera(self):
        """カメラを自動検出して初期化"""
        success, message = self.camera_device.initializeCamera()
        
        if success:
            QMessageBox.information(self, "Camera Initialized", message)
            # ステータス表示を更新
            camera_type = self.camera_device.getCameraType().value
            self.widget_manager.dict_label["status_display"].setText(
                f"Status: {camera_type} Camera Ready"
            )
        else:
            QMessageBox.critical(self, "Camera Error", message)
            self.widget_manager.dict_label["status_display"].setText(
                "Status: No Camera"
            )
            sys.exit(0)
    

    
    """
    makeLayout Function; Component
    小要素のLayout
    return -> Layout
    """
    
    def makeLayoutComponentSaveDirectory(self):
        """Save Directory, Prefix, Move Destination の設定"""
        layout = QVBoxLayout()
        
        # Save Directory
        layout.addLayout(makeLayoutLoadFileWidget(
            self.widget_manager,
            label="Save Directory:",
            key_label="savedir_label",
            key_lineedit="savedir",
            key_button="savedir_browse",
            text_set=f"{os.getcwd().replace(os.sep, '/')}",
            axis="horizontal"
        ))
        
        # Save Directory Prefix
        layout.addLayout(makeLayoutComboBoxLabel(
            self.widget_manager,
            key_combobox="savedir_prefix",
            key_label="savedir_prefix_label",
            label="Save Directory Prefix:",
            items=["", "dlc-pupil", "HoG-ActiveWhisking", "HoG-Sniffing"],
            idx_default=0,
            axis="horizontal"
        ))
        
        # Move Destination
        layout.addLayout(makeLayoutLoadFileWidget(
            self.widget_manager,
            label="Move Destination:",
            key_label="movedst_label",
            key_lineedit="movedst",
            key_button="movedst_browse",
            text_set="Z:/database",
            axis="horizontal"
        ))
        
        return layout
    
    def makeLayoutComponentCameraConfig(self):
        """Camera Config (fps, width, height, offset, gain, exposure)"""
        layout = QVBoxLayout()
        
        # セクションタイトル
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="camera_config_title",
            label="Camera Settings",
            bold=True,
            use_global_style=False
        ))
        
        # カメラパラメータ
        params = [
            ("fps", "FPS:", "60.0"),
            ("width", "Width:", "1280"),
            ("height", "Height:", "1024"),
            ("offsetx", "Offset X:", "0"),
            ("offsety", "Offset Y:", "0"),
            ("gain", "Gain:", "12.0"),
            ("exposure_time", "Exposure Time:", "2000"),
        ]
        
        for key, label_text, default_value in params:
            layout_param = makeLayoutLineEditLabel(
                self.widget_manager,
                key_lineedit=f"camera_{key}",
                key_label=f"camera_{key}",
                label=label_text,
                text_set=default_value,
                axis="horizontal"
            )
            layout_param.addStretch()
            layout.addLayout(layout_param)
        
        return layout
    
    def makeLayoutComponentCapturePreview(self):
        """Capture Single shot, Play, FPS表示"""
        layout = QVBoxLayout()
        
        # ボタン行
        layout_buttons = QHBoxLayout()
        layout_buttons.addWidget(self.widget_manager.makeWidgetButton(
            key="capture_single",
            label="Capture Single",
            use_global_style=True
        ))
        layout_buttons.addWidget(self.widget_manager.makeWidgetButton(
            key="play",
            label="Play",
            use_global_style=True
        ))
        layout_buttons.addStretch()
        layout.addLayout(layout_buttons)
        
        # FPS表示
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="fps_display",
            label="FPS: 0",
            use_global_style=True
        ))
        
        return layout
    
    def makeLayoutComponentCaptureControl(self):
        """Start Capture with Bpod, Move Video Files, Exit"""
        layout = QVBoxLayout()
        
        # 撮影開始ボタン
        layout.addWidget(self.widget_manager.makeWidgetButton(
            key="start_capture_bpod",
            label="Start Capture with Bpod",
            use_global_style=True
        ))
        
        # ステータス表示
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="status_display",
            label="Status: Standby",
            use_global_style=True
        ))
        
        # ファイル移動ボタン
        layout.addWidget(self.widget_manager.makeWidgetButton(
            key="move_video",
            label="Move Video Files",
            use_global_style=True
        ))
        
        # 終了ボタン
        layout.addWidget(self.widget_manager.makeWidgetButton(
            key="exit",
            label="Exit",
            use_global_style=True
        ))
        
        return layout
    
    def makeLayoutComponentImageDisplay(self):
        """カメラ画像表示用キャンバス"""
        layout = QVBoxLayout()
        
        # 画像表示エリア
        label_image = self.widget_manager.makeWidgetLabel(
            key="camera_preview",
            label="Camera Preview Area",
            align=Qt.AlignCenter,
            use_global_style=True
        )
        label_image.setMinimumSize(1280, 1024)
        label_image.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)
        label_image.setStyleSheet("border: 1px solid black; background-color: #f0f0f0;")
        layout.addWidget(label_image, stretch=1)
        
        return layout
    
    """
    makeLayout Function; Section
    領域レベルの大Layout
    """
    # Left Upper Section
    def makeLayoutSectionLeftUpper(self):
        """Config全体（Save Directory + Camera Config）"""
        layout = QVBoxLayout()
        
        # title
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="config_title",
            label="Configuration",
            font_size=14,
            bold=True,
            use_global_style=False
        ))
        
        # コンポーネント
        layout.addLayout(self.makeLayoutComponentSaveDirectory())
        layout.addLayout(self.makeLayoutComponentCameraConfig())
        layout.addStretch()
        
        return layout
    
    # Left Lower Section
    def makeLayoutSectionLeftLower(self):
        """Capture全体（Preview + Control）"""
        layout = QVBoxLayout()
        
        # タイトル
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="capture_title",
            label="Capture Control",
            font_size=14,
            bold=True,
            use_global_style=False
        ))
        
        # コンポーネント
        layout.addLayout(self.makeLayoutComponentCapturePreview())
        layout.addLayout(self.makeLayoutComponentCaptureControl())
        layout.addStretch()
        
        return layout
    
    # Right Section
    def makeLayoutSectionRight(self):
        """Image表示"""
        layout = QVBoxLayout()
        
        # タイトル
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="image_title",
            label="Camera View",
            font_size=14,
            bold=True,
            use_global_style=False
        ))
        
        # コンポーネント
        layout.addLayout(self.makeLayoutComponentImageDisplay())
        
        return layout
    
    """
    bindFunc Function
    配置したwidgetに関数を紐づけ
    """
    def bindFuncAllWidget(self):
        bindFuncExit(q_window=self, q_button=self.widget_manager.dict_button["exit"])

        for key_button in ["savedir_browse", "movedst_browse"]:
            bindFuncLoadFileWidget(
                q_button=self.widget_manager.dict_button[key_button],
                q_widget=self,
                q_lineedit=self.widget_manager.dict_lineedit[key_button.replace("_browse", "")],
                filetype=None,
                select_dir=True
            )

        # Capture Single
        bindFuncCaptureSingle(
            q_button=self.widget_manager.dict_button["capture_single"],
            camera_device=self.camera_device,
            display_engine=self.display_engine
        )


# 実行用
if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
    applyAppStyle(app)
    gui = BehaviorCameraGUI()
    gui.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\HyperPC_Smee\anaconda3\envs\camera\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [20]:
gui.camera_device.captureFrame()

In [1]:
# behavior_camera/gui/bind_func.py
# カメラ操作用のbindFunc関数

from PyQt5.QtWidgets import QPushButton, QLabel
from PyQt5.QtCore import QTimer


def bindFuncCaptureSingle(
    q_button: QPushButton,
    camera_device: 'CameraDevice',
    display_engine: 'DisplayEngine'
) -> None:
    """
    Capture Singleボタンにキャプチャ関数をバインド
    
    Args:
        q_button: Capture Singleボタン
        camera_device: CameraDeviceインスタンス
        display_engine: DisplayEngineインスタンス
    """
    def captureSingle():
        if not camera_device.isInitialized():
            return
        
        # 1フレームキャプチャ
        frame = camera_device.captureFrame()
        if frame is not None:
            display_engine.updateDisplay(frame)
    
    q_button.clicked.connect(captureSingle)


def bindFuncPlay(
    q_button: QPushButton,
    q_label_fps: QLabel,
    capture_session: 'CaptureSession',
    display_engine: 'DisplayEngine',
    parent
) -> None:
    """
    Playボタンに連続キャプチャ（プレビュー）機能をバインド
    
    Args:
        q_button: Playボタン
        q_label_fps: FPS表示用ラベル
        capture_session: CaptureSessionインスタンス
        display_engine: DisplayEngineインスタンス
        parent: 親ウィジェット（GUIインスタンス）
    """
    # タイマーとフラグを親に保存
    parent.play_timer = QTimer()
    parent.fps_update_timer = QTimer()
    parent.fps_frame_count = 0
    
    def updateFrame():
        """フレームを更新"""
        # フレーム取得
        frame = capture_session.getFrame()
        if frame is not None:
            display_engine.updateDisplay(frame)
            parent.fps_frame_count += 1
    
    def updateFPS():
        """FPS表示を更新"""
        fps = parent.fps_frame_count
        q_label_fps.setText(f"FPS: {fps}")
        parent.fps_frame_count = 0
    
    def startPlay():
        """プレビュー開始"""
        # 撮影開始
        if not capture_session.startCapture():
            return
        
        # タイマー開始
        parent.play_timer.timeout.connect(updateFrame)
        parent.play_timer.start(16)  # 約60fps
        
        # FPS更新タイマー開始（1秒ごと）
        parent.fps_update_timer.timeout.connect(updateFPS)
        parent.fps_update_timer.start(1000)
        
        q_button.setText("Stop")
    
    def stopPlay():
        """プレビュー停止"""
        # タイマー停止
        parent.play_timer.stop()
        parent.fps_update_timer.stop()
        
        # 撮影停止
        capture_session.stopCapture()
        
        parent.fps_frame_count = 0
        q_button.setText("Play")
        q_label_fps.setText("FPS: 0")
        
        # 平均FPS表示
        avg_fps = capture_session.getAverageFPS()
        print(f"Average FPS: {avg_fps:.2f}")
    
    def togglePlay():
        """プレビューのトグル"""
        if capture_session.isCapturing():
            stopPlay()
        else:
            startPlay()
    
    q_button.clicked.connect(togglePlay)